# 1. Save Model

In [1]:
import tensorflow as tf
%matplotlib inline

print(tf.__version__)

2.4.2


In [2]:
! nvidia-smi

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
Thu Sep  9 10:17:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   48C    P0    33W /  70W |      0MiB / 15109MiB |      0%    

## 1.1 Binary Classification Model

In [3]:
from crack_utils import classification_block
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.regularizers import l2

def MLP(input_size):
    visible = Input(shape=(input_size, ))
    layer = classification_block(visible, 16, decay=l2(0.001))
    model = Model(inputs=visible, outputs=layer)
    return model

### 1.1.1 DenseNet201 (Highest Accuracy)

```
Test loss: 0.28
Test Accuracy: 0.90
Test Precision: 0.92
Test True Positive Rate: 0.89
```

In [4]:
from crack_utils import load_binary_dataset, extract_in_batch
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input as pre_densenet

model = DenseNet201()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

SRC_BINARY = "/home/ubuntu/crack/Origin-Binary-Full"

x_train, _, y_train, _ = load_binary_dataset(SRC_BINARY, (224, 224))
x_train = extract_in_batch(20, x_train, model, "train", pre_densenet)

Extension horovod.torch has not been built: /home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-37m-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2021-09-09 10:17:29.423 ip-172-31-5-51:10428 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-09-09 10:17:29.445 ip-172-31-5-51:10428 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


Number of samples for training:

In [5]:
x_train.shape

(6770, 1920)

Train and save the model:

```
Epoch 00092: early stopping
```

In [6]:
%%time

from crack_utils import binary_compile

model = MLP(x_train.shape[1])
binary_compile(model, 0.001, 0.9)
model.fit(x=x_train, y=y_train, batch_size=64, epochs=90, verbose=0)
model.save("model/binary_TL_DenseNet201.h5")

CPU times: user 1min 3s, sys: 9.61 s, total: 1min 13s
Wall time: 27.3 s


## 1.2 Multinomial Classification Model

### 1.2.1 ResNet152V2 (Highest Accuracy)

```
Test loss: 1.12
Test accuracy: 0.63
Test f1 micro: 0.63
Test f1 macro: 0.55
Test f1 weighted: 0.60
```

In [7]:
def MLP(input_size):
    visible = Input(shape=(input_size, ))
    layer = classification_block(visible, 32, classes=5, decay=l2(0.001))
    model = Model(inputs=visible, outputs=layer)
    return model

In [8]:
from crack_utils import load_multi_dataset
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input as pre_resnet_v2

SRC_MULTI = "/home/ubuntu/crack/Origin-Multi-Full"

model = ResNet152V2()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

del x_train, y_train
x_train, _, y_train, _ = load_multi_dataset(SRC_MULTI, (224, 224))

x_train = extract_in_batch(10, x_train, model, "train", pre_resnet_v2)

# convert integers to dummy variables (i.e. one hot encoded)
y_train = to_categorical(y_train)

Number of samples for training:

In [9]:
x_train.shape

(3385, 2048)

Train and save the model:

```
Epoch 00022: early stopping
```

Train and save the models:

In [10]:
%%time

from crack_utils import multiple_compile

model = MLP(x_train.shape[1])
multiple_compile(model, 0.001, 0.9)
model.fit(x=x_train, y=y_train, batch_size=64, epochs=25, verbose=0)
model.save("model/multi_TL_ResNet152V2.h5")

CPU times: user 3.78 s, sys: 556 ms, total: 4.34 s
Wall time: 2.42 s


### 1.2.2 DenseNet201 (Highest Macro F1 Score)

```
Test loss: 0.96
Test accuracy: 0.60
Test f1 micro: 0.60
Test f1 macro: 0.59
Test f1 weighted: 0.60
```

In [11]:
def MLP(input_size):
    visible = Input(shape=(input_size, ))
    layer = classification_block(visible, 32, classes=5)
    model = Model(inputs=visible, outputs=layer)
    return model

In [12]:
model = DenseNet201()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

del x_train, y_train
x_train, _, y_train, _ = load_multi_dataset(SRC_MULTI, (224, 224))

x_train = extract_in_batch(10, x_train, model, "train", pre_densenet)

# convert integers to dummy variables (i.e. one hot encoded)
y_train = to_categorical(y_train)

Train and save the models:

In [13]:
%%time

model = MLP(x_train.shape[1])
multiple_compile(model, 0.01, 0.9)
model.fit(x=x_train, y=y_train, batch_size=64, epochs=30, verbose=0)
model.save("model/multi_TL_DenseNet201.h5")

CPU times: user 4.12 s, sys: 613 ms, total: 4.74 s
Wall time: 2.66 s
